In [7]:
import vk
import pandas as pd
import random
import time
from sklearn.externals import joblib
from scipy.sparse import csr_matrix
from pathlib import Path

## Function to get movies info from VK

In [5]:
def get_data(id, n, min_num_of_movies=3):
    movies = pd.Series()
    last_id = 0
    index = 0
    count = 0
    
    while movies.size < n:
        
        
        f = vk.get_friends(str(id))

        friends = []
        for x in f:
            friends.append(str(x))
            
        for friend in friends:
            if movies.size >= n:
                break
            
            try:
                user_info = vk.get_users_info([friend])

                if count == 100:
                    print("pause")
                    time.sleep(1)
                    count = 0
                else:
                    count += 1

                if ("movies" in user_info[friend].keys() 
                    and len(user_info[friend]["movies"]) >= min_num_of_movies
                    and not friend in movies.keys()
                    and len(friends) >= 5):
                    if index % 20 == 0:
                        joblib.dump(movies, filename="movies", compress=9)
                    movies.set_value(index, user_info[friend]["movies"].split(","))
                    index += 1
                    print(friend)
            except:
                pass
        print("Index: " + str(index))
        print("Id: " + str(id))
        print("Last id: " + str(last_id))
        if movies.size < n:
            last_id = id
            while True: 
                ind = random.randint(0, len(friends))
                if (len(vk.get_friends(friends[ind])) != 0
                    or last_id != friends[ind]):
                    id = friends[ind]
                    break
            
    return movies

## Function to get unique movies

In [3]:
def get_unique_movies(data):
    unique_movies = pd.Series()
    index = 0
    for movies in data:
        for movie in movies:
            if not movie in unique_movies:
                unique_movies.set_value(index, movie)
                index += 1
    return unique_movies

## Function to construct a matrix

In [4]:
def construct_matrix(data, unique_movies):
    row = []
    column = []

    for user in data.keys():
        for movie in data[user]:
            row.append(user)
            column.append(unique_movies[unique_movies == movie].index[0])
    data = [1 for x in range(len(row))]
    movies_matrix = csr_matrix((data, (row, column)), \
                               shape=(len(data), len(unique_movies)))
    return movies_matrix

# Main stuff

In [9]:
file = Path("movies")
if file.is_file():
    data = joblib.load("movies")
else:
    data = get_data(1, 1000)

In [10]:
unique_movies = get_unique_movies(data)

In [11]:
matrix = construct_matrix(data, unique_movies)